In [1]:
import sys, os #imp
sys.path.append('..')
import pandas as pd
import yaml
import backtesting as bt
import matplotlib.pyplot as plt

from tools.instruments import instruments

In [2]:
%load_ext autoreload
%aimport backtesting.system
%aimport backtesting.trader
%aimport tools.quotes

%autoreload 2

In [3]:
plt.ioff()

In [4]:
with open('systems/sample.yaml', encoding='utf-8') as f:
    system1 = yaml.load(f, Loader=yaml.FullLoader)
    print(system1)

{'name': 'testSystem', 'description': '시스템 설명', 'sectors': 'default', 'instruments': ['AD'], 'from_date': '2019-08-01', 'to_date': None, 'principal': 1000000, 'heat_system': 'DefaultHeat', 'max_system_heat': 0.3, 'max_sector_heat': 0.3, 'max_trade_heat': 0.01, 'max_lots': 100, 'commission': 5.98, 'skid': 0.25, 'metrics': [['ema30', 'EMA', 'window=30'], ['ema5', 'EMA', 'window=5'], ['min30', 'MIN', 'window=30'], ['max300', 'MAX', 'window=300']], 'entry_rule': {'long': 'ema5>ema30', 'short': None}, 'exit_rule': {'long': 'ema5<ema30', 'short': None}, 'stop_rule': {'long': 'min30', 'short': None}}


In [5]:
trader = bt.Trader(systems=[system1])
system = trader.systems[0]
#trader.run()

In [13]:
system.create_report()

{'name': 'testSystem', 'description': '시스템 설명', 'sectors': 'default', 'instruments': ['AD'], 'from_date': '2019-08-01', 'to_date': None, 'principal': 1000000, 'heat_system': 'DefaultHeat', 'max_system_heat': 0.3, 'max_sector_heat': 0.3, 'max_trade_heat': 0.01, 'max_lots': 100, 'commission': 5.98, 'skid': 0.25, 'metrics': [['ema30', 'EMA', 'window=30'], ['ema5', 'EMA', 'window=5'], ['min30', 'MIN', 'window=30'], ['max300', 'MAX', 'window=300']], 'entry_rule': {'long': 'ema5>ema30', 'short': None}, 'exit_rule': {'long': 'ema5<ema30', 'short': None}, 'stop_rule': {'long': 'min30', 'short': None}}
ema5<ema30
<html>

<body align="center">
    <div>
        <h1> 시스템 성능 보고서 </h1>
        <h3> 테스트 날짜: 2023년 06월 25일</h3>
        <div align="center">
            <table border="1px solid black" width="800px" style="border-collapse:collapse">
                <tr>
                    <th width="20%"> 시스템명</th>
                    <td style="padding-left:10px"> testSystem</td>
                </tr>


In [119]:
system.metrics.attrs

{'type': {'ema30': 'price',
  'ema5': 'price',
  'min30': 'price',
  'max300': 'price'},
 'name': 'testSystem'}

In [10]:
df=system.metrics['AD']['min30']

In [12]:
df.index

DatetimeIndex(['1959-08-03', '1959-08-04', '1959-08-05', '1959-08-06',
               '1959-08-07', '1959-08-10', '1959-08-11', '1959-08-12',
               '1959-08-13', '1959-08-14',
               ...
               '2022-04-21', '2022-04-22', '2022-04-25', '2022-04-26',
               '2022-04-27', '2022-04-28', '2022-04-29', '2022-05-02',
               '2022-05-03', '2022-05-04'],
              dtype='datetime64[ns]', name='date', length=15987, freq=None)

In [13]:
system.symbols

['AD']

In [16]:
starts = {} #첫 거래 시작일
for symbol in ['AD']:
    flag = trader.quotes[symbol][['open','high','low','close']].isna().any(axis=1)
    system.signals.loc[flag, symbol] = np.nan
    starts[symbol] = trader.quotes[symbol][~flag].index[0]

In [17]:
starts

{'AD': Timestamp('1987-01-13 00:00:00')}

In [19]:
flag = trader.quotes['AD'].isna().any(axis=1)

In [27]:
system.signals.first_valid_index()

Timestamp('1987-01-13 00:00:00')

In [31]:
system.signals.index[system.signals.index.get_loc(system.signals.first_valid_index())-1]

Timestamp('1987-01-12 00:00:00')

In [38]:
df=df.iloc[1000]

In [47]:
mask = df.isna().groupby('symbol').any() 

In [51]:
mask[~mask]

symbol
BO    False
C     False
HG    False
W     False
Name: 1963-07-22 00:00:00, dtype: bool

In [158]:
f=d =eval("df['ema30']>df['min30']")

In [159]:
f.name=('AD','stop_rule')

In [161]:
pd.concat([d,f],axis=1)

AD          
           stop_rule stop_rule
date                          
1959-08-03     False     False
1959-08-04     False     False
1959-08-05     False     False
1959-08-06     False     False
1959-08-07     False     False
...              ...       ...
2022-04-28      True      True
2022-04-29      True      True
2022-05-02      True      True
2022-05-03      True      True
2022-05-04      True      True

[15987 rows x 2 columns]

In [111]:
df

field,open,high,low,close,volume,oi
date,,,,,,
1959-08-03,0.00000,0.00000,0.00000,0.00000,0.0,0.0
1959-08-04,0.00000,0.00000,0.00000,0.00000,0.0,0.0
1959-08-05,0.00000,0.00000,0.00000,0.00000,0.0,0.0
1959-08-06,0.00000,0.00000,0.00000,0.00000,0.0,0.0
1959-08-07,0.00000,0.00000,0.00000,0.00000,0.0,0.0
...,...,...,...,...,...,...
2022-04-28,0.71290,0.71660,0.70595,0.70995,112019.0,144283.0
2022-04-29,0.71035,0.71845,0.70610,0.70870,124501.0,144179.0
2022-05-02,0.70710,0.70870,0.70355,0.70415,91807.0,140428.0


In [94]:
mask

date
1959-08-03     True
1959-08-04     True
1959-08-05     True
1959-08-06     True
1959-08-07     True
              ...  
2022-04-28    False
2022-04-29    False
2022-05-02    False
2022-05-03    False
2022-05-04    False
Length: 16028, dtype: bool

In [101]:
instruments['AD'].quotes()

field,open,high,low,close,volume,oi
date,,,,,,
1987-01-13,0.01070,0.01180,0.00680,0.00700,1854,915
1987-01-14,-0.00300,-0.00190,-0.00640,-0.00530,1446,930
1987-01-15,0.00480,0.00490,0.00350,0.00370,539,786
1987-01-16,0.00560,0.00680,0.00560,0.00650,537,964
1987-01-19,0.00640,0.00980,0.00640,0.00830,506,866
...,...,...,...,...,...,...
2022-04-28,0.71290,0.71660,0.70595,0.70995,112019,144283
2022-04-29,0.71035,0.71845,0.70610,0.70870,124501,144179
2022-05-02,0.70710,0.70870,0.70355,0.70415,91807,140428


In [49]:
quote = trader.quotes

In [52]:
quote.MA(window=30, inplace=True)

In [65]:
df=Quotes(quote['AD'], type='single')

In [67]:
df.MA(window=10, inplace=True)

In [68]:
df

field,open,high,low,close,volume,oi,ma30,ma10
date,,,,,,,,
1959-08-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1959-08-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1959-08-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1959-08-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1959-08-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-04-28,0.71290,0.71660,0.70595,0.70995,112019.0,144283.0,0.741827,0.727695
2022-04-29,0.71035,0.71845,0.70610,0.70870,124501.0,144179.0,0.740832,0.724340
2022-05-02,0.70710,0.70870,0.70355,0.70415,91807.0,140428.0,0.739580,0.721195


In [73]:
df.columns.levels

AttributeError: 'Index' object has no attribute 'levels'

In [75]:
f=quote.columns

In [88]:
len(f.names)

2

In [81]:
d =df.columns

In [87]:
len(d.names)

1